In [2]:
%run Functions.py

In [3]:
def LingeringOnKeys(userKeys):
    
    timeLingeringOnKeysList = list()
    
    keyPrevious = userKeys[0]
    
    keySelected = 'Sleep'
    
    for i, userKey in enumerate(userKeys):
        
        if float(userKey[2]) == 1:
            keySelected = userKey[1]
            keyPrevious = userKey
            
            continue
        
        if keySelected != userKey[1]:
            keyPrevious = userKey
            continue
        
        if keyPrevious[1] != userKey[1]:
            keyPrevious = userKey
            continue
        
        # Proceeding here means keyPrevious and keyCurrent are the same
        if len(keyPrevious[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
            
        if len(userKey[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
        
        time1, t1, t2 = userKey[0].partition('+')
        timeCurrent = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        keyPrevious = userKey
        
        if i <= len(userKeys) - 1:
            if userKey[1] == userKeys[i+1][1]:
                if float(userKeys[i+1][2]) > 0:
                    continue
                else:
                    timeLingeringOnKeysList.append([timeCurrent, float(userKey[3])/1000])
                    print(userKey)
                    keySelected = 'Sleep'
            else:
                if float(userKey[2]) == 0:
                    #timeLingeringOnKeysList.append([timeCurrent, 250])
                    print('not adding 250', userKey)
                else:
                    timeLingeringOnKeysList.append([timeCurrent, float(userKey[3])/1000])
                    print(userKey)
                keySelected = 'Sleep'
    
        
    
    return timeLingeringOnKeysList  

In [4]:
metricComputed_mean = 'TimeRatioLingeringOnKeys'
metricComputed_std = 'TimeLingeringOnKeysStd'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_mean = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed_mean +  '.xlsx'
resultFileName_std = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed_std +  '.xlsx'


for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'af\\1' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                       
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None or scratchPad is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find time between selection of 1 key and start of selection of the next
            TimeLingeringOnKeysList = LingeringOnKeys(userKeys_wDwellTime)
            
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, \
                                                   userKeys_wDwellTime)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            TimeLingeringOnKeysTrialsMean = MetricForEveryTrial(TimeLingeringOnKeysList, \
                                                                    eventTrialsInKeysSelected_writing)
            
            
            if '1stPart' in root:
                print('1stPart')
                TimeLingeringOnKeysTrialsMean1 = TimeLingeringOnKeysTrialsMean
                #TimeLingeringOnKeysTrialsStd1 = TimeLingeringOnKeysTrialsStd
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                TimeLingeringOnKeysTrialsMean2 = TimeLingeringOnKeysTrialsMean
                #TimeLingeringOnKeysTrialsStd2 = TimeLingeringOnKeysTrialsStd
                
                TimeLingeringOnKeysTrialsMean = TimeLingeringOnKeysTrialsMean1 + TimeLingeringOnKeysTrialsMean2
                #TimeLingeringOnKeysTrialsStd = TimeLingeringOnKeysTrialsStd1 + TimeLingeringOnKeysTrialsStd2
                
                TimeLingeringOnKeysTrialsMean1 = list()
                #TimeLingeringOnKeysTrialsStd1 = list()
                
                
            dataToSave_mean = DataForEveryTrial()
            dataToSave_mean.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_mean.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_mean.sessionNumber = subjAndSessionName[-1]
            dataToSave_mean.variable = metricComputed_mean
            dataToSave_mean.dataForTrial = TimeLingeringOnKeysTrialsMean
            dataToSave_mean.resultPathName = resultFileName_mean
            
            """
            dataToSave_std = DataForEveryTrial()
            dataToSave_std.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_std.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_std.sessionNumber = subjAndSessionName[-1]
            dataToSave_std.variable = metricComputed_std
            dataToSave_std.dataForTrial = TimeLingeringOnKeysTrialsStd
            dataToSave_std.resultPathName = resultFileName_std
            
            """
            print(dataToSave_mean.printInfo())
            #print(dataToSave_std.printInfo())
            
            
            #dataToSave_mean.AddToFile()
            #dataToSave_std.AddToFile()
            
            
            
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
['2019-02-11T11:18:38.0918399+01:00', 'd', 0.0408605, '32.6884']
['2019-02-11T11:19:35.2547368+01:00', 'a', 0.01375125, '11.001']
['2019-02-11T11:19:41.8271987+01:00', 'g', 0.041142125, '32.9137']
['2019-02-11T11:20:10.7646117+01:00', 'r', 0.054855, '32.913000000000004']
['2019-02-11T11:20:33.7521751+01:00', 'g', 0.0565296666666667, '33.91780000000002']
['2019-02-11T11:20:52.2004049+01:00', '4', 0.0747955, '44.8773']
['2019-02-11T11:21:33.2072525+01:00', 'i', 0.313682166666667, '188.20930000000018']
['2019-02-11T11:21:38.2930289+01:00', 'i', 0.129191333333333, '77.5147999999998']
['2019-02-11T11:21:49.5085391+01:00', 'r', 0.111883, '67.1298']
['2019-02-11T11:22:41.6741875+01:00', 't', 0.0555646666666667, '33.33880000000002']
['2019-02-11T11:23:08.0708498+01:00', '4', 0.3692145, '221.52870000000001']
['2019-02-11T11:26:40.1106884+01:00', 's', 0.0199461666666667, '11.9677000000000

[0.0109711, 0.010845800000000001, 0.2572813]
109.258923
  subjectID block session  trial  TimeRatioLingeringOnKeys
0        ac     2       2      0                  0.000000
1        ac     2       2      1                  0.000000
2        ac     2       2      2                  0.000000
3        ac     2       2      3                  0.000000
4        ac     2       2      4                  0.000851
subject path E:\Data\Data\ac\3_MS\2019-02-14-14-28-49_1
subject and session name:  ac__3_MS__2019-02-14-14-28-49_1
['2019-02-14T14:29:01.0076250+01:00', 't', 0.014962125, '11.9697']
['2019-02-14T14:29:15.8751711+01:00', 'u', 0.1372065, '109.76520000000001']
['2019-02-14T14:33:58.2817979+01:00', 'o', 0.194546, '155.6368']
not adding 250 ['2019-02-14T14:34:17.2993721+01:00', 'o', 0.0, '0.0']
['2019-02-14T14:34:28.9480469+01:00', 'r', 0.59640625, '477.125']
['2019-02-14T14:35:51.8018409+01:00', 'f', 0.011224625, '8.9797']
['2019-02-14T14:36:06.8203767+01:00', 'a', 0.137398375, '109.9186

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-b6c8f0c48493>", line 153, in <module>
    userKeys_wDwellTime)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 735, in FindReadingPartsOfTrial_inKeysSelected
    userKeysTime = timeConversion(userKeysTimeStr)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 556, in timeConversion
    timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 577, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 404, in _strptime
    day = int(found_dict['d'])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Trac

KeyboardInterrupt: 